<br>
<h1 style = "font-size:35px; font-family:cursive ; font-weight : bold; color : #AA3073; text-align: center; border-radius: 10px 100px;">Zomato Bangalore Restaurants 🍜🥂 </h1>
<br>

![](https://im.whatshot.in/img/2018/Dec/skyee-is-1545640792.jpg)

**Bangalore** officially known as Bengaluru is the capital and the largest city of the Indian state of Karnataka. It has a population of more than 8 million and a metropolitan population of around 11 million, making it the third most populous city and fifth most populous urban agglomeration in India.Located in southern India on the Deccan Plateau, at a height of over 900 m (3,000 ft) above sea level, Bangalore is known for its pleasant climate throughout the year. Its elevation is the highest among the major cities of India.
Bangalore is widely regarded as the "Silicon Valley of India" (or "IT capital of India") because of its role as the nation's leading information technology (IT) exporter.Indian technological organisations are headquartered in the city. A demographically diverse city, Bangalore is the second fastest-growing major metropolis in India.

**Zomato** is an Indian multinational restaurant aggregator and food delivery company founded by Pankaj Chaddah and Deepinder Goyal in 2008.Zomato provides information, menus and user-reviews of restaurants as well as food delivery options from partner restaurants in select cities.As of 2019, the service is available in 24 countries and in more than 10,000 cities.

### About the Data:
- url : This feature contains the url of the restaurant on the Zomato website
- address : This feature contains the address of the restaurant in Bangalore
- name : This feature contains the name of the restaurant
- online_order : whether online ordering is available in the restaurant or not
- book_table : table book option available or not
- rate : contains the overall rating of the restaurant out of 5
- votes : contains total number of upvotes for the restaurant
- phone : contains the phone number of the restaurant
- location : contains the neighborhood in which the restaurant is located
- rest_type : restaurant type
- dish_liked : dishes people liked in the restaurant
- cuisines : food styles, separated by comma
- approx_cost(for two people) : contains the approximate cost of meal for two people
- reviews_list : list of tuples containing reviews for the restaurant, each tuple consists of two values, rating and review by the customer
- menu_item : contains list of menus available in the restaurant
- listed_in(type) : type of meal
- listed_in(city) : contains the neighborhood in which the restaurant is located

In [ ]:
from IPython.display import HTML
HTML("""
<style>
h1,h2,h3 {
	margin: 1em 0 0.5em 0;
	font-weight: 600;
	font-family: 'Titillium Web', sans-serif;
	position: relative;  
	font-size: 36px;
	line-height: 40px;
	padding: 15px 15px 15px 2.5%;
	color: #13003A;
	box-shadow: 
		inset 0 0 0 1px rgba(53,86,129, 1), 
		inset 0 0 5px rgba(53,86,129, 1),
		inset -285px 0 35px white;
	border-radius: 0 10px 0 15px;
	background: #fff
    
}
</style>
""")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">✅ Importing Required Libraries</h1>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objs as go
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
from plotly import tools
import re
from plotly.offline import init_notebook_mode, plot, iplot
from wordcloud import WordCloud, STOPWORDS 
from warnings import filterwarnings
filterwarnings('ignore')

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">🔍Basic Data Exploration:</h1>

In [ ]:
df=pd.read_csv("../input/zomato-bangalore-restaurants/zomato.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.dropna(axis=0,subset=['location'],inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
len(df['location'].unique())

In [ ]:
locations=pd.DataFrame({"Name":df['location'].unique()})

In [ ]:
locations.head()

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">🗺 Geospatial Analysis:</h1>

In [ ]:
geolocator=Nominatim(user_agent="app")

In [ ]:
lat=[]
lon=[]
for location in locations['Name']:
    location = geolocator.geocode(location)    
    if location is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(location.latitude)
        lon.append(location.longitude)

In [ ]:
locations['lat']=lat
locations['lon']=lon

In [ ]:
locations.head()

In [ ]:
Rest_locations=pd.DataFrame(df['location'].value_counts().reset_index())

In [ ]:
Rest_locations.columns=['Name','count']
Rest_locations.head()

In [ ]:
print(locations.shape)
print(Rest_locations.shape)

In [ ]:
Restaurant_locations=Rest_locations.merge(locations,on='Name',how="left").dropna()
Restaurant_locations.head()

In [ ]:
Restaurant_locations['count'].max()

In [ ]:
#### Bengaluru Coordinates - (12.9716° N, 77.5946° E)
def generateBaseMap(default_location=[12.97, 77.59], default_zoom_start=12):
    base_map = folium.Map(location=default_location, zoom_start=default_zoom_start)
    return base_map

In [ ]:

basemap=generateBaseMap()

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Base Map for Bengaluru:</h1>

In [ ]:
basemap

In [ ]:
Restaurant_locations[['lat','lon','count']]

In [ ]:
HeatMap(Restaurant_locations[['lat','lon','count']],zoom=20,radius=15).add_to(basemap)

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Heatmap for Restaurants in Bengaluru:</h1>

In [ ]:
basemap

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold"> Marker Cluster Map for Restaurants in Bengaluru:</h1>

In [ ]:
FastMarkerCluster(data=Restaurant_locations[['lat','lon','count']].values.tolist()).add_to(basemap)

basemap

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Marker Cluster Map according to average ratings of Restaurants:</h1>

In [ ]:
df['rate'].unique()

In [ ]:
df.dropna(axis=0,subset=['rate'],inplace=True)

In [ ]:
df['rate'].unique()

In [ ]:
def split(x):
    return x.split('/')[0]

In [ ]:
df['rating']=df['rate'].apply(split)
df['rating'].unique()

In [ ]:
df.replace('NEW',0,inplace=True)
df.replace('-',0,inplace=True)

In [ ]:
df['rating']=pd.to_numeric(df['rating'])
df.groupby(['location'])['rating'].mean().sort_values(ascending=False)

In [ ]:
df.groupby(['location'])['rating'].mean()

In [ ]:
avg_rating=df.groupby(['location'])['rating'].mean().values

In [ ]:
loc=df.groupby(['location'])['rating'].mean().index

In [ ]:
geolocator=Nominatim(user_agent="app")

In [ ]:
lat=[]
lon=[]
for location in loc:
    location = geolocator.geocode(location)    
    if location is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(location.latitude)
        lon.append(location.longitude)

In [ ]:
rating=pd.DataFrame()
rating['location']=loc
rating['lat']=lat
rating['lon']=lon
rating['avg_rating']=avg_rating

In [ ]:
rating.head()

In [ ]:
rating.isna().sum()

In [ ]:
rating=rating.dropna()

In [ ]:
HeatMap(rating[['lat','lon','avg_rating']],zoom=20,radius=15).add_to(basemap)
basemap

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Heatmap for 'North Indian' Cuisines:</h1>

In [ ]:
df2= df[df['cuisines']=='North Indian']
df2.head()

In [ ]:
north_india=df2.groupby('location')['url'].count().reset_index()
north_india.columns=['Name','count']
north_india.head()

In [ ]:
north_india=north_india.merge(locations,on="Name",how='left').dropna()

In [ ]:
basemap=generateBaseMap()
HeatMap(north_india[['lat','lon','count']].values.tolist(),zoom=20,radius=15).add_to(basemap)
basemap

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Heatmap for 'South Indian' Cuisines:</h1>

In [ ]:
def Heatmap_Zone(zone):
    df3=df[df['cuisines']==zone]
    df_zone=df3.groupby(['location'],as_index=False)['url'].agg('count')
    df_zone.columns=['Name','count']
    df_zone=df_zone.merge(locations,on="Name",how='left').dropna()
    basemap=generateBaseMap()
    HeatMap(df_zone[['lat','lon','count']].values.tolist(),zoom=20,radius=15).add_to(basemap)
    return basemap

In [ ]:
df['cuisines'].unique()

In [ ]:
Heatmap_Zone('South Indian')

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Heatmap for 'Italian' Cuisines:</h1>

In [ ]:
Heatmap_Zone('Italian')

<h1 style="background-color:#FF85A3;font-size:25px;color:#00033E;font-weight : bold">Some More EDA:</h1>

In [ ]:
df.shape

In [ ]:
df2=df.copy()

In [ ]:
# df.isnull().sum()
# df.dropna(how='any',inplace=True)
df = df.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type',
                                  'listed_in(city)':'city'})
df.columns

In [ ]:
df['cost'] = df['cost'].astype(str)
df['cost'] = df['cost'].apply(lambda x: x.replace(',','.')) 
df['cost'] = df['cost'].astype(float)

In [ ]:
df.name = df.name.apply(lambda x:x.title())
df.online_order.replace(('Yes','No'),(True, False),inplace=True)
df.book_table.replace(('Yes','No'),(True, False),inplace=True)

In [ ]:
df.shape

In [ ]:
def Encode(data):
    for column in data.columns[~data.columns.isin(['rate', 'cost', 'votes','rating'])]:
        data[column] = data[column].factorize()[0]
    return data

Zom_df = Encode(df.copy())
Zom_df.head()

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Is Online delivering available?</h1>

In [ ]:
labels = ["Accepted",'Not Accepted']
values = df['online_order'].value_counts()
colors = ['mediumslateblue', 'darkorange']
fig = go.Figure(data=[go.Pie(labels=labels,
                             values=values,hole=.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=3)))
fig.update_layout(title="Online delivering available? ",
                  titlefont={'size': 30},      
                  )
fig.show()

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Is Table booking available?</h1>

In [ ]:
labels = ["Accepted",'Not Accepted']
values = df['book_table'].value_counts()
colors = ['mediumslateblue', 'darkorange']
fig = go.Figure(data=[go.Pie(labels=labels,
                             values=values,hole=.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=3)))
fig.update_layout(title="Table booking available? ",
                  titlefont={'size': 30},
                  )
fig.show()

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Most popular cuisines of Bangalore </h1>

In [ ]:
values = df['cuisines'].value_counts()[:20]
labels=values.index
text=values.index
fig = go.Figure(data=[go.Pie(values=values,labels=labels,hole=.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=3)))
fig.update_layout(title="Most popular cuisines of Bangalore ",
                  titlefont={'size': 30},
                  )
fig.show()

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Cost comparison for Online orders V/s Offline orders:</h1>

In [ ]:
fig = px.box(df,x='online_order',y='cost',color='online_order')

fig.update_layout(title="Cost comparison for Online order",
                  titlefont={'size': 30},template='simple_white'
                  )
fig.show()

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">WordCloud for Dishes in each Restaurant Type:</h1>

In [ ]:
dfupd=df.copy()
dfupd['update_dish_liked']=dfupd['dish_liked'].apply(lambda x : x.split(',') if type(x)==str else [''])
rest=dfupd['rest_type'].value_counts()[:9].index

In [ ]:
dfupd.dropna(axis='index',subset=['rest_type'],inplace=True)
dfupd.dropna(axis='index',subset=['dish_liked'],inplace=True)

In [ ]:
def produce_wordcloud(rest):
    
    plt.figure(figsize=(20,30))
    for i,restaurant in enumerate(rest):
        plt.subplot(3,3,i+1)
        dishes=''
        data=dfupd[dfupd['rest_type']==restaurant]
        for word in data['dish_liked']:
            words=word.split()
            # Converts each token into lowercase 
            for i in range(len(words)): 
                words[i] = words[i].lower() 
            dishes=dishes+ " ".join(words)+" "
        wordcloud = WordCloud(max_font_size=None, background_color='white', collocations=False,stopwords = stopwords,width=1200, height=1200).generate(dishes)
        plt.imshow(wordcloud)
        plt.title(restaurant)
        plt.axis("off")

In [ ]:
stopwords = set(STOPWORDS) 
produce_wordcloud(rest)

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">WordCloud for reviews:</h1>

In [ ]:
def reviewwords(restaurant):
    dataset=dfupd[dfupd['rest_type']==restaurant]
    total_review=' '
    for review in dataset['reviews_list']:
        review=review.lower()
        review=re.sub('[^a-zA-Z]', ' ',review)
        review=re.sub('rated', ' ',review)
        review=re.sub('x',' ',review)
        review=re.sub(' +',' ',review)
        total_review=total_review + str(review)
    wordcloud = WordCloud(width = 800, height = 800, 
            background_color ='white', 
            stopwords = set(STOPWORDS), 
            min_font_size = 10).generate(total_review) 
    # plot the WordCloud image                        
    plt.figure(figsize = (8, 8)) 
    plt.imshow(wordcloud) 
    plt.axis("off")

In [ ]:
reviewwords('Quick Bites')

In [ ]:
reviewwords('Cafe')

In [ ]:
reviewwords('Delivery')

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Count of total Restaurants in each Locations:</h1>


In [ ]:
fig=px.bar(x=df['city'].unique(),y=df['city'].value_counts(),labels=dict(x="City Name",y="Total Count"),color_continuous_scale="Agsunset",color=df['city'].unique())
fig.update_layout(title="Location wise counts for Restaurants ",
                  titlefont={'size': 30},template='simple_white'     
                  )
fig.update_traces(marker_line_color='black',
                  marker_line_width=2, opacity=1)
fig.show()

In [ ]:
df2=df.copy()
df2.dropna(how='any',inplace=True)

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Loaction wise ratings of Restaurants: </h1>

In [ ]:
loc_plt=pd.crosstab(df2['rating'],df2['city'])
fig=px.bar(loc_plt,x=loc_plt.index,y=loc_plt.columns,barmode='stack',opacity=1)
fig.update_layout(title="Location wise Rating",
                  titlefont={'size': 30},
                  template='simple_white'       
                  )
fig.update_traces(marker_line_color='black',
                  marker_line_width=0.5, opacity=0.8)
fig.show()

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Number of Restaurants based on Services:</h1>

In [ ]:
fig=px.histogram(df['type'],labels=dict(value='Type'))
fig.update_layout(title="Type of Services",
                  titlefont={'size': 30},template='simple_white'     
                  )
fig.update_traces(marker_color='darkorange', marker_line_color='black',
                  marker_line_width=2, opacity=1)

fig.show()

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Count of Restaurants based on the Cost</h1>

In [ ]:
fig=px.histogram(df['cost'],labels=dict(value='Cost Range'),nbins=10)
fig.update_layout(title="Cost of Restaurants",
                  titlefont={'size': 30},template='simple_white'     
                  )
fig.update_traces(marker_color='darkorange', marker_line_color='black',
                  marker_line_width=2, opacity=1)

fig.show()

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Top 10 popular Restaurant Chains in Bengaluru</h1>

In [ ]:
chains=df['name'].value_counts()[:10]
fig=px.bar(y=chains,x=chains.index,labels=dict(x='Name',y='Count'),color_continuous_scale="Agsunset",color=chains.index)
fig.update_layout(title="Most famous restaurant chains",
                  titlefont={'size': 30},template='simple_white'     
                  )
fig.update_traces( marker_line_color='black',
                  marker_line_width=2, opacity=1)

fig.show()

In [ ]:
df2.isnull().sum()

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">👨‍💻Modeling with Pycaret:</h1>

In [ ]:
!pip install pycaret

In [ ]:
train=Zom_df.iloc[:,[3,4,6,8,9,11,12,14,17]]
train.head()

In [ ]:
from pycaret.regression import *

In [ ]:
exp = setup(data = train, target = 'rating', session_id=101,silent=True)

In [ ]:
models()

In [ ]:
top3 = compare_models(n_select = 3,exclude = ['xgboost','catboost'])

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">Visualizing Model Performance:</h1>

In [ ]:
rf = create_model('rf', fold = 5,verbose=False)
et = create_model('et', fold = 5,verbose=False)
dt = create_model('dt', fold = 5,verbose=False)

In [ ]:
plot_model(rf,plot='residuals')

In [ ]:
plot_model(rf,plot='error')

In [ ]:
plot_model(rf,plot='feature')

In [ ]:
plot_model(rf,plot='parameter')

In [ ]:
plot_model(et,plot='residuals')

In [ ]:
plot_model(et,plot='error')

In [ ]:
plot_model(dt,plot='residuals')

In [ ]:
plot_model(dt,plot='error')

![](https://media.tenor.com/images/218f408b75e6fd13324cbd1fe3981295/tenor.gif)

# **Do upvote if you like my work**